<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py -O preprocess_sku.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-11-10 21:49:25--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  19.7MB/s    in 0.3s    

2022-11-10 21:49:26 (19.7 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-11-10 21:49:26--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess_sku.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5052 (4.9K) [text/plain]
Saving to: ‘preprocess_sku.py’

preprocess_sku.py   100%[===================>]

In [ ]:
# ! pip install -q deepmatch

In [2]:
import pandas as pd
from ast import literal_eval
from preprocess_sku import gen_data_set, gen_model_input, gen_data_set_timesplit
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 4.4 MB/s 
time: 1.12 ms (started: 2022-11-10 21:49:39 +00:00)


In [5]:
import gc
gc.collect()

109

time: 166 ms (started: 2022-11-10 21:49:39 +00:00)


# Data

In [ ]:
# data_path = "./"

# unames = ['user_id','gender','age','occupation','zip']
# user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
# rnames = ['user_id','sku_number','rating','timestamp']
# ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
# mnames = ['sku_number','title','genres']
# movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
# movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

# data = pd.merge(pd.merge(ratings,movies),user).iloc[:10000] # taking 1w
# print(data.shape)
# print(data.user_id.nunique())
# print(data.sku_number.nunique())
# data.head(2)

time: 1.18 ms (started: 2022-10-28 18:23:51 +00:00)


### Sales data

In [6]:
data = pd.read_csv('sample_data/mik_sales_28_7.csv', index_col = False, 
                   parse_dates = ['trans_date','created_time']
                   )
# data['most_recent_purchased_sku'] = data['most_recent_purchased_sku'].apply(literal_eval)
# data['most_recent_view_sku'] = data['most_recent_view_sku'].apply(literal_eval)
# data['hist_sku_number'] = data['most_recent_purchased_sku'] + data['most_recent_view_sku']
data.shape

(731745, 10)

time: 2.31 s (started: 2022-11-10 22:01:15 +00:00)


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731745 entries, 0 to 731744
Data columns (total 10 columns):
 #   Column                              Non-Null Count   Dtype         
---  ------                              --------------   -----         
 0   data_source                         731745 non-null  object        
 1   user_id                             731745 non-null  int64         
 2   sku_number                          731745 non-null  object        
 3   qty                                 731745 non-null  int64         
 4   trans_date                          731745 non-null  datetime64[ns]
 5   created_time                        731745 non-null  datetime64[ns]
 6   email_address                       731745 non-null  object        
 7   geo_zip                             731308 non-null  object        
 8   most_frequent_device_class_general  731735 non-null  object        
 9   category_path                       731745 non-null  object        
dtypes: datet

In [8]:
data.shape, data['user_id'].nunique(), data['sku_number'].nunique()

((731745, 10), 236769, 41045)

time: 132 ms (started: 2022-11-10 22:01:21 +00:00)


#### Take use sample 2000

In [9]:
user_sp = pd.read_csv('sample_data/user_sp_28_7.csv', index_col = False)
user_sp.head(2)

,user_id
0,86461517433732359
1,-8467210055931700772


time: 11.8 ms (started: 2022-11-10 22:01:25 +00:00)


In [10]:
data = data[data['user_id'].isin(list(user_sp['user_id'].values))]
data.shape, data['user_id'].nunique(), data['sku_number'].nunique()

((54890, 10), 2000, 14778)

time: 40.9 ms (started: 2022-11-10 22:01:28 +00:00)


In [11]:
data['trans_date'].max(),data['trans_date'].min(), data['trans_date'].max() - data['trans_date'].min()

(Timestamp('2022-03-31 00:00:00'),
 Timestamp('2022-02-24 00:00:00'),
 Timedelta('35 days 00:00:00'))

time: 5.84 ms (started: 2022-11-10 22:01:31 +00:00)


In [12]:
import datetime
# test_start_date = pd.Timestamp(datetime.datetime.today().date() - datetime.timedelta(6))
test_start_date = pd.Timestamp(data['trans_date'].max().date() - datetime.timedelta(6))

test_start_date

Timestamp('2022-03-25 00:00:00')

time: 5.48 ms (started: 2022-11-10 22:01:34 +00:00)


In [13]:
data[(data['trans_date'] < test_start_date)].shape, data[(data['trans_date'] >= test_start_date)].shape

((36188, 10), (18702, 10))

time: 14.5 ms (started: 2022-11-10 22:01:40 +00:00)


In [ ]:
data = data[['data_source', 'user_id', 'sku_number', 'qty', 'trans_date',
       'created_time', 'email_address', 'geo_zip','most_frequent_device_class_general','category_path']]

time: 13.4 ms (started: 2022-11-02 19:49:08 +00:00)


In [19]:
data['sku_number_org'] = data['sku_number']
data['user_id_org'] = data['user_id']

time: 3.26 ms (started: 2022-11-10 22:01:57 +00:00)


### Item category mapping dat

In [ ]:
df_item = pd.read_csv("sample_data/mik_item_07292022.csv", index_col=False)
df_item.shape

In [ ]:
df_item.head(2)

In [ ]:
data['most_recent_view_sku'].apply(lambda x: len(x)).describe()

# Feature Engineering

In [14]:
sparse_features = ['sku_number', 'category_path',
                   'user_id', 'geo_zip','most_frequent_device_class_general']

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


time: 1.55 ms (started: 2022-11-10 22:01:44 +00:00)


In [15]:
# view_item_set = list(set(data['most_recent_view_sku'].sum()))
# full_item_set = list(set(list(data['sku_number'].unique()) + view_item_set))
# len(full_item_set)

time: 1.17 ms (started: 2022-11-10 22:01:47 +00:00)


In [16]:
feature_max_idx = {}
for feature in sparse_features:
    # if feature == 'sku_number':      
    #   lbe_sku = LabelEncoder()
    #   lbe_sku.fit(full_item_set)
    #   data[feature] = lbe_sku.transform(data[feature]) + 1
    #   feature_max_idx[feature] = data[feature].max() + 1
    # else:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

time: 123 ms (started: 2022-11-10 22:01:47 +00:00)


In [17]:
feature_max_idx

{'sku_number': 14779,
 'category_path': 1006,
 'user_id': 2001,
 'geo_zip': 1619,
 'most_frequent_device_class_general': 3}

time: 5.56 ms (started: 2022-11-10 22:01:48 +00:00)


In [20]:
user_profile = data[['user_id', 'geo_zip', 'most_frequent_device_class_general','user_id_org']].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["sku_number", 'sku_number_org']].drop_duplicates('sku_number')
user_profile.shape, item_profile.shape

((2000, 3), (14778, 2))

time: 19.8 ms (started: 2022-11-10 22:01:59 +00:00)


In [ ]:
user_profile.to_csv('sample_data/user_profile_28_7.csv', index=True)
item_profile.to_csv('sample_data/item_profile_28_7.csv', index=True)

time: 52.4 ms (started: 2022-11-02 19:49:23 +00:00)


In [21]:
# user_item_list = data.groupby("user_id")['sku_number'].apply(list)
user_item_list = data.groupby("user_id")['sku_number'].apply(list)

user_item_list.head()

user_id
1    [926, 10105, 4404, 3405, 821, 5794, 3435, 5793...
2    [3054, 7307, 14683, 5714, 14694, 6844, 7847, 1...
3    [5480, 4949, 7035, 4950, 5477, 5474, 5457, 279...
4    [10302, 10290, 9960, 2797, 2485, 3315, 2766, 1...
5    [7884, 12207, 14659, 14658, 14657, 14656, 9767...
Name: sku_number, dtype: object

time: 66.2 ms (started: 2022-11-10 22:02:11 +00:00)


In [22]:
train_set, test_set = gen_data_set_timesplit(data, test_start_date, SEQ_LEN, NUMBER_NEG_SAMPLE)
len(train_set), len(test_set)

100%|██████████| 2000/2000 [00:08<00:00, 242.63it/s]


7 7


(69190, 18295)

time: 8.38 s (started: 2022-11-10 22:02:13 +00:00)


In [ ]:
data[data['trans_date'] < test_start_date].shape, data[data['trans_date'] >= test_start_date].shape

((36188, 12), (18702, 12))

time: 18.7 ms (started: 2022-11-02 20:10:40 +00:00)


In [ ]:
test_start_date

Timestamp('2022-03-25 00:00:00')

time: 10.6 ms (started: 2022-11-02 20:12:31 +00:00)


In [ ]:
a = pd.DataFrame(train_set)
b = pd.DataFrame(test_set)
a.shape, b.shape

((69190, 9), (18295, 9))

time: 305 ms (started: 2022-11-02 20:13:46 +00:00)


In [ ]:
a[(a[0]==841) & (a[2]==1)].shape

(21, 9)

time: 9.76 ms (started: 2022-11-02 21:31:02 +00:00)


In [ ]:
train = data[data['created_time'] < test_start_date]

time: 13.7 ms (started: 2022-11-02 21:30:08 +00:00)


In [ ]:
set(train[train['user_id'] == 841]['sku_number'].values) - set(a[(a[0]==841) & (a[2]==1)][1].values)

{6806}

time: 18.8 ms (started: 2022-11-02 21:35:43 +00:00)


In [ ]:
train[(train['user_id'] == 841) & (train['sku_number'] == 6806)]

,data_source,user_id,sku_number,qty,trans_date,created_time,email_address,geo_zip,most_frequent_device_class_general,category_path,sku_number_org,user_id_org
343133,ADOBE,841,6806,1,2022-02-27,2022-02-27 13:40:32,cortnerae01@gmail.com,1493,1,369,10591594,-1544467516583031877


time: 29.3 ms (started: 2022-11-02 21:36:19 +00:00)


In [ ]:
data.user_id.nunique()

2000

time: 5.55 ms (started: 2022-11-02 19:49:31 +00:00)


In [ ]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

time: 1.26 ms (started: 2022-11-02 19:49:31 +00:00)


In [26]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess_sku.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((69190,), (69190,), (18295,), (18295,))

time: 1.05 s (started: 2022-11-10 22:04:21 +00:00)


In [29]:
train_model_input

{'user_id': array([ 150, 1577, 1095, ..., 1067,  847,  496]),
 'sku_number': array([13559,  3138, 12617, ...,  6536,  8267,  3877]),
 'hist_sku_number': array([[13549,  4138,  9222, ...,     0,     0,     0],
        [ 3137, 13667,  3139, ...,     0,     0,     0],
        [ 5745,  5744,  7509, ...,     0,     0,     0],
        ...,
        [ 1762, 10964,  5346, ...,     0,     0,     0],
        [ 5749,  8959, 12150, ...,     0,     0,     0],
        [ 7300,  7296,  7295, ...,     0,     0,     0]], dtype=int32),
 'hist_category_path': array([[  71,  751,  853, ...,    0,    0,    0],
        [  96,  102,   96, ...,    0,    0,    0],
        [ 433,  433,  432, ...,    0,    0,    0],
        ...,
        [  15,  342,  388, ...,    0,    0,    0],
        [ 377,  390,  340, ...,    0,    0,    0],
        [1001, 1001, 1001, ...,    0,    0,    0]], dtype=int32),
 'hist_len': array([11, 29, 18, ...,  9, 26, 20]),
 'category_path': array([ 71,  96,  55, ..., 666, 246, 230]),
 'geo_zip

time: 16.8 ms (started: 2022-11-10 22:05:54 +00:00)


In [ ]:
test_model_input['user_id'].shape

(18295,)

time: 3.42 ms (started: 2022-11-02 19:49:32 +00:00)


In [ ]:
test_model_input['user_id'].shape

(18295,)

time: 4.5 ms (started: 2022-11-02 19:49:32 +00:00)


In [30]:
pd.DataFrame(train_set, columns = ['user_id', 'item_id','label','hist_item_id','seq_len',
                                   'hist_genres', 'genres']).head(5).to_csv('sample_data/train_set_28_7.csv')

time: 163 ms (started: 2022-11-10 22:07:11 +00:00)


In [ ]:
pd.DataFrame(test_set, columns = ['user_id', 'item_id','label','hist_item_id','seq_len',
                                   'hist_genres', 'genres']).to_csv('sample_data/test_set_28_7.csv')

time: 285 ms (started: 2022-11-02 19:49:33 +00:00)


# Create Embeddings

In [ ]:
from collections import Counter
train_counter = Counter(train_model_input['sku_number'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['sku_number'])]
len(train_counter), type(train_counter), len(item_count)

(14515, collections.Counter, 14779)

time: 23.1 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

time: 2.71 ms (started: 2022-11-02 19:49:34 +00:00)


### Create Feature Embeddings



In [ ]:
# item features
item_feature_columns = ['sku_number']
# User features
user_feature_columns = ['user_id', 'geo_zip', 'most_frequent_device_class_general',                        
                        'hist_sku_number',
                        'hist_category_path',
                        'hist_len'
                        ]


time: 2.3 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
feature_max_idx

{'sku_number': 14779,
 'category_path': 1006,
 'user_id': 2001,
 'geo_zip': 1619,
 'most_frequent_device_class_general': 3}

time: 5.01 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
user_sparse_feature_columns = user_feature_columns[0:3]
user_sparse_feature_columns

['user_id', 'geo_zip', 'most_frequent_device_class_general']

time: 5.4 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
user_seq_sparse_feature_columns = user_feature_columns[3:-1]
user_seq_sparse_feature_columns 

['hist_sku_number', 'hist_category_path']

time: 4.01 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


time: 4.94 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

time: 28.1 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

time: 3.25 ms (started: 2022-11-02 19:49:34 +00:00)


In [ ]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b6fc210>,
 'geo_zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b76fa10>,
 'most_frequent_device_class_general': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b175590>,
 'hist_sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b2cfed0>,
 'hist_category_path': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b175ad0>,
 'sku_number': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7f526b2cfd10>}

time: 4.68 ms (started: 2022-11-02 19:49:34 +00:00)


# Create User Inputs


## Create Inputs Tensor

In [ ]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

time: 25.7 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'geo_zip:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'most_frequent_device_class_general:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_sku_number:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_category_path:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len:0' shape=(None, 1) dtype=int32>]

time: 4.88 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


time: 6.14 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
item_inputs_list

[<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>]

time: 4.44 ms (started: 2022-11-02 19:49:35 +00:00)


## Embed Inputs

In [ ]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

time: 41.5 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

time: 42 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

time: 1.48 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

time: 2.16 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_geo_zip/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_most_frequent_device_class_general/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_1/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

time: 5.01 ms (started: 2022-11-02 19:49:35 +00:00)


## Combine Embeded Input (User only)

In [ ]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

time: 1.95 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten/Reshape:0' shape=(None, 112) dtype=float32>

time: 11.1 ms (started: 2022-11-02 19:49:35 +00:00)


# Sepecify Model Layers

### User Layers

In [ ]:
from tensorflow.keras import layers

time: 1.71 ms (started: 2022-11-02 19:49:35 +00:00)


In [ ]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

time: 2.29 ms (started: 2022-11-02 19:49:36 +00:00)


In [ ]:
# from deepctr.layers import DNN
from model_utils import DNN

time: 2.24 ms (started: 2022-11-02 19:49:36 +00:00)


In [ ]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



time: 2.85 ms (started: 2022-11-02 19:49:36 +00:00)


In [ ]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 429 ms (started: 2022-11-02 19:49:36 +00:00)


### Item Layer

In [ ]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_1/l2_normalize:0' shape=(14779, 32) dtype=float32>

time: 400 ms (started: 2022-11-02 19:49:36 +00:00)


In [ ]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(14779, 32)

time: 288 ms (started: 2022-11-02 19:49:37 +00:00)


In [ ]:
# .eval(session=K.get_session())

time: 1.55 ms (started: 2022-11-02 19:49:37 +00:00)


### Output Layer

In [ ]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer/ExpandDims:0' shape=(None, 1) dtype=float32>

time: 197 ms (started: 2022-11-02 19:49:37 +00:00)


In [ ]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

time: 1.33 ms (started: 2022-11-02 19:49:37 +00:00)


In [ ]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

time: 1.58 ms (started: 2022-11-02 19:49:38 +00:00)


In [ ]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_1/l2_normalize:0' shape=(14779, 32) dtype=float32>

time: 4.98 ms (started: 2022-11-02 19:49:38 +00:00)


In [ ]:
user_dnn_output

<tf.Tensor 'lambda/l2_normalize:0' shape=(None, 32) dtype=float32>

time: 4.45 ms (started: 2022-11-02 19:49:38 +00:00)


In [ ]:
item_features[item_feature_name]

<tf.Tensor 'sku_number:0' shape=(None, 1) dtype=int32>

time: 5.48 ms (started: 2022-11-02 19:49:38 +00:00)


## Build Model

In [ ]:
from tensorflow.python.keras.models import Model

time: 1.75 ms (started: 2022-11-02 19:49:38 +00:00)


In [ ]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


time: 85.4 ms (started: 2022-11-02 19:49:38 +00:00)


# Model Compile and Training

In [ ]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

time: 340 ms (started: 2022-11-02 19:49:38 +00:00)


In [ ]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 69190 samples
Epoch 1/20
69190/69190 [==============================] - 10s 141us/sample - loss: 4.8023
Epoch 2/20
69190/69190 [==============================] - 11s 157us/sample - loss: 4.6962
Epoch 3/20
69190/69190 [==============================] - 9s 129us/sample - loss: 4.6569
Epoch 4/20
69190/69190 [==============================] - 9s 136us/sample - loss: 4.5661
Epoch 5/20
69190/69190 [==============================] - 11s 162us/sample - loss: 4.5181
Epoch 6/20
69190/69190 [==============================] - 9s 128us/sample - loss: 4.4610
Epoch 7/20
69190/69190 [==============================] - 10s 147us/sample - loss: 4.4419
Epoch 8/20
69190/69190 [==============================] - 9s 128us/sample - loss: 4.4079
Epoch 9/20
69190/69190 [==============================] - 9s 129us/sample - loss: 4.3778
Epoch 10/20
69190/69190 [==============================] - 9s 132us/sample - loss: 4.3758
Epoch 11/20
69190/69190 [==============================] - 9s 130us/sample - loss:

In [ ]:
res = model.predict(test_model_input)
res.shape

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


time: 11.2 s (started: 2022-11-02 19:52:51 +00:00)


# Predict: Generate Item and User Embeddings

In [ ]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"sku_number": item_profile['sku_number'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(18295, 32)
(14778, 32)
time: 321 ms (started: 2022-11-02 19:53:02 +00:00)


In [ ]:
train_user_embs = user_embedding_model.predict(train_model_input, batch_size=2 ** 12)


time: 577 ms (started: 2022-11-02 20:51:47 +00:00)


In [ ]:
# import numpy as np
# np.save('sample_data/user_embs.npy', user_embs)
# np.save('sample_data/item_embs.npy', item_embs)

time: 2.08 ms (started: 2022-11-02 19:53:49 +00:00)


In [ ]:
prob_matrix = user_embs.dot(item_embs.T)
prob_matrix.shape

# Recommendation

In [ ]:
! pip install faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 8.6 MB 6.5 MB/s 
time: 10.4 s (started: 2022-11-02 19:54:05 +00:00)


In [ ]:
import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

time: 35.4 ms (started: 2022-11-02 19:54:15 +00:00)


In [ ]:
def recall_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / len(y_true)

time: 1.89 ms (started: 2022-11-02 19:54:15 +00:00)


In [ ]:
def precision_N(y_true, y_pred, N=30):
    return len(set(y_pred[:N]) & set(y_true)) * 1.0 / N

time: 1.71 ms (started: 2022-11-02 19:54:15 +00:00)


In [ ]:
test_true_label = {line[0]:[line[1]] for line in test_set}

import numpy as np
import faiss
from tqdm import tqdm
# from deepmatch.utils import recall_N

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)
D, I = index.search(np.ascontiguousarray(user_embs), 50)
recall = []
precision = []
hit = 0
pred_label = {}
for i, uid in tqdm(enumerate(test_user_model_input['user_id'])):
    try:
        pred = [item_profile['sku_number'].values[x] for x in I[i]]
        filter_item = None
        recall_score = recall_N(test_true_label[uid], pred, N=30)
        precision_score = precision_N(test_true_label[uid], pred, N=30)

        recall.append(recall_score)
        precision.append(precision_score)
        pred_label[uid] = pred
        if test_true_label[uid] in pred:
            hit += 1
    except:
        print(i)
print("")
print("recall", np.mean(recall))
print("precision", np.mean(precision))
print("hit rate", hit / len(test_user_model_input['user_id']))

18295it [00:14, 1253.91it/s]


recall 0.0705657283410768
precision 0.0023521909447025598
hit rate 0.10614922109866083
time: 16.2 s (started: 2022-11-02 19:54:15 +00:00)


In [ ]:
result_rec = pd.DataFrame.from_dict(pred_label, orient='index').reset_index()

time: 98 ms (started: 2022-11-02 19:54:32 +00:00)


In [ ]:
result_rec.to_csv('sample_data/result_rec_2000.csv')

time: 46.9 ms (started: 2022-11-02 19:54:32 +00:00)


# Validation

In [ ]:
df_train = pd.DataFrame(train_set)

In [ ]:
test_user_model_input['user_id'][0:10]

In [ ]:
user_idx = 0
uid = 1854

In [ ]:
D[user_idx]

In [ ]:
sorted_index = prob_matrix.argsort()[:,-50:]
-np.sort(-prob_matrix[user_idx][sorted_index[user_idx]])

In [ ]:
I[user_idx]

In [ ]:
(sorted_index[user_idx])[::-1]

In [ ]:
np.array([item_profile['sku_number'].values[x] for x in I[user_idx]])

In [ ]:
len(set(df_train[df_train[0]==uid][1].unique()) & set([item_profile['sku_number'].values[x] for x in I[user_idx]]))

In [ ]:
len(set(df_train[df_train[0]==uid][1].unique()) & set(sorted_index[user_idx]))

In [ ]:
item_profile['sku_number']

In [ ]:
all_item_model_input